In [194]:
import pandas as pd
import os
import json
import re
from tqdm import tqdm # to show progress bar in for loops

# to increase display width of dataframes in jupyter notebook
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

In [2]:
root = "movie_dataset/"

In [3]:
os.listdir(root)

['movies_metadata.csv',
 'keywords.csv',
 'ratings.csv',
 'credits.csv',
 'links.csv',
 'links_small.csv',
 'ratings_small.csv']

## credits.csv

In [4]:
credits = pd.read_csv(root+'credits.csv')

In [5]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [6]:
credits.shape
credits.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [186]:
def repl_call(qbody):
    m = qbody.sea
    preq = m.group(1)
    qbody = m.group(2)
    qbody = re.sub( r'(?<=[^"])"(?=[^"])', ' ', qbody )
    qbody = re.sub( r'"', '', qbody )
    return preq + '"' + qbody + '"'

def str_to_json_v2(df, col, row):
    regex = r"\"[a-zA-Z_\"0-9 ]*( )*:( )*(\"[a-zA-Z_\"0-9 ]*\"|[0-9]+)"
    regex2 = r":( )*(\"[a-zA-Z_\"0-9 ]*\"|[0-9]+)"

    crew00= df[col][row].replace("\\","").replace("\'","\"").replace("None","\"\"")
    crew00 = split_with_delimiter(crew00, '}, ', '}')
    crew00[0] = crew00[0][1:]
    crew00[-1] = crew00[-1][:-2]

    all_files = []
    for i in range(len(crew00)):
        try:
            all_files.append(json.loads(crew00[i]))
        except ValueError:
            new_string = "{"
            matches = re.finditer(regex, crew00[i])
            for match in matches:
                key_val = match.group()
                matches2 = re.finditer(regex2, key_val)
                for match2 in matches2:
                    endposn = match2.start()
                    val = match2.group()
                    val = val.replace(":", "").strip()
                    if val[0]=="\"" and val[-1]=="\"":
                        val = val[1:-1].replace("\"", "\'")
                        val = " \"" + val + "\""
                    else:
                        val = " " + val

                    new_string+= key_val[:endposn+1]+val
                new_string+=' ,'

            new_string = new_string[:-2]
            new_string+='}'
            all_files.append(json.loads(new_string))
    
    return all_files

def split_with_delimiter(text, delimiter, repl):

    text = text.split(delimiter)
    text =  [e+repl for e in text if e]
    return text

def run_csv(df, no_rows=2):
    credits_json = []
    for x in tqdm(range(2)):
        tmp_dict = {}
        for y in df.columns:
            #print(y)
            if y  == 'id':
                tmp_dict['id'] = df[y][x]
            else:
                tmp_dict[y] = str_to_json_v2(df,y,x)

        credits_json.append(tmp_dict)
    return credits_json

def json_to_file(obj, fp='test.json', indent=3):
    with open('test.json', 'w') as f:
        json.dump(obj, f, default=str, indent=indent)
    print('file saved',fp)

In [187]:
lol = run_csv(credits)

100%|██████████| 2/2 [00:00<00:00, 1654.23it/s]


In [188]:
json_to_file(lol)

file saved test.json


## movies_metadata.csv

In [189]:
movies_meta = pd.read_csv(root+'movies_metadata.csv')

/home/reverie-pc/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [211]:
movies_meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [216]:
movies_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [210]:
movies_meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


#### from the above inspection, we can see that we don't have any use for the following columns: video

In [206]:
def clean_genres(x):
    '''
    To show only the clear genre names as a list
    '''
    x = x.replace('\'', '\"')
    return [gen['name'] for gen in json.loads(x)]

In [209]:
movies_meta['genres'] = movies_meta['genres'].apply(clean_genres)

In [213]:
movies_meta['belongs_to_collection'][0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

### links.csv

In [217]:
links_df = pd.read_csv(os.path.join(root, 'links.csv'))

In [218]:
links_df

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
45838,176269,6209470,439050.0
45839,176271,2028550,111109.0
45840,176273,303758,67758.0
45841,176275,8536,227506.0
